# mTAN - Imputation Model

Here we will test out the mTAN imputation model

In [31]:
import sys
path = '../../toy_dataset/'
sys.path.append(path)

from data_utils import ToyDataset

In [ ]:
path_dataset = '/home2/joshua.wendland/Documents/sepsis/toy_dataset/synthetic_ts_1/synthetic_ts_test_data_eav.csv.gz'

# Import, Import, Import

In [1]:
import argparse
import numpy as np
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

from random import SystemRandom
import src.models
import src.utils

ModuleNotFoundError: No module named 'physionet'

# Parse arguments

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--niters', type=int, default=2000, help='Number of epochs')
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--std', type=float, default=0.01)
parser.add_argument('--latent-dim', type=int, default=32)
parser.add_argument('--rec-hidden', type=int, default=32)
parser.add_argument('--gen-hidden', type=int, default=50)
parser.add_argument('--embed-time', type=int, default=128)
parser.add_argument('--k-iwae', type=int, default=10)
parser.add_argument('--save', type=int, default=1)
parser.add_argument('--enc', type=str, default='mtan_rnn')
parser.add_argument('--dec', type=str, default='mtan_rnn')
parser.add_argument('--fname', type=str, default=None)
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--n', type=int, default=8000)
parser.add_argument('--batch-size', type=int, default=50)
parser.add_argument('--quantization', type=float, default=0.016,
                    help="Quantization on the physionet dataset.")
parser.add_argument('--classif', action='store_true',
                    help="Include binary classification loss")
parser.add_argument('--norm', action='store_true')
parser.add_argument('--kl', action='store_true')
parser.add_argument('--learn-emb', action='store_true')
parser.add_argument('--enc-num-heads', type=int, default=1)
parser.add_argument('--dec-num-heads', type=int, default=1)
parser.add_argument('--length', type=int, default=20)
parser.add_argument('--num-ref-points', type=int, default=128)
parser.add_argument('--dataset', type=str, default='toy')
parser.add_argument('--enc-rnn', action='store_false')
parser.add_argument('--dec-rnn', action='store_false')
parser.add_argument('--sample-tp', type=float, default=1.0)
parser.add_argument('--only-periodic', type=str, default=None)
parser.add_argument('--dropout', type=float, default=0.0)
args = parser.parse_args()

# Set up Dataset

In [ ]:
# Set random seeds
experiment_id = int(SystemRandom().random() * 100000)
print(args, experiment_id)
seed = args.seed
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

# Set the device to run calculation on
device = torch.device(
    'cuda' if torch.cuda.is_available() else 'cpu')

# consider the correct dataset
if args.dataset == 'toy':
    data_obj = utils.kernel_smoother_data_gen(args, alpha=100., seed=0)

train_loader = data_obj["train_dataloader"]
test_loader = data_obj["test_dataloader"]
dim = data_obj["input_dim"]

# Set up model

In [ ]:
if args.enc == 'enc_rnn3':
    rec = models.enc_rnn3(
        dim, torch.linspace(0, 1., args.num_ref_points), args.latent_dim, 
        args.rec_hidden, 128, learn_emb=args.learn_emb).to(device)
elif args.enc == 'mtan_rnn':
    rec = models.enc_mtan_rnn(
        dim, torch.linspace(0, 1., args.num_ref_points), args.latent_dim, args.rec_hidden, 
        embed_time=128, learn_emb=args.learn_emb, num_heads=args.enc_num_heads).to(device)

    
if args.dec == 'rnn3':
    dec = models.dec_rnn3(
        dim, torch.linspace(0, 1., args.num_ref_points), args.latent_dim, 
        args.gen_hidden, 128, learn_emb=args.learn_emb).to(device)
elif args.dec == 'mtan_rnn':
    dec = models.dec_mtan_rnn(
        dim, torch.linspace(0, 1., args.num_ref_points), args.latent_dim, args.gen_hidden, 
        embed_time=128, learn_emb=args.learn_emb, num_heads=args.dec_num_heads).to(device)


    

# Set up optimizer, tensorboard, load model weights (and more?)

In [ ]:
params = (list(dec.parameters()) + list(rec.parameters()))
optimizer = optim.Adam(params, lr=args.lr)
print('parameters:', utils.count_parameters(rec), utils.count_parameters(dec))
if args.fname is not None:
    checkpoint = torch.load(args.fname)
    rec.load_state_dict(checkpoint['rec_state_dict'])
    dec.load_state_dict(checkpoint['dec_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print('loading saved weights', checkpoint['epoch'])
    print('Test MSE', utils.evaluate(dim, rec, dec, test_loader, args, 1))
    print('Test MSE', utils.evaluate(dim, rec, dec, test_loader, args, 3))
    print('Test MSE', utils.evaluate(dim, rec, dec, test_loader, args, 10))
    print('Test MSE', utils.evaluate(dim, rec, dec, test_loader, args, 20))
    print('Test MSE', utils.evaluate(dim, rec, dec, test_loader, args, 30))
    print('Test MSE', utils.evaluate(dim, rec, dec, test_loader, args, 50))


# Set up Tensorboard
start_time = datetime.now().strftime("%Y.%m.%d.%H.%M.%S")
path = '/home2/joshua.wendland/Documents/sepsis/imputation/mTAN/runs/'
path += f'{args.dataset}/'
path += f'/{start_time}'
writer = SummaryWriter(log_dir=path)

# Train through epochs

In [ ]:
# Run through epochs
for itr in range(1, args.niters + 1):
    train_loss = 0
    train_n = 0
    avg_reconst, avg_kl, mse = 0, 0, 0
    if args.kl:
        wait_until_kl_inc = 10
        if itr < wait_until_kl_inc:
            kl_coef = 0.
        else:
            kl_coef = (1 - 0.99 ** (itr - wait_until_kl_inc))
    else:
        kl_coef = 1

    for train_batch in train_loader:
        train_batch = train_batch.to(device)
        batch_len = train_batch.shape[0]
        observed_data = train_batch[:, :, :dim]
        observed_mask = train_batch[:, :, dim:2 * dim]
        observed_tp = train_batch[:, :, -1]
        if args.sample_tp and args.sample_tp < 1:
            subsampled_data, subsampled_tp, subsampled_mask = utils.subsample_timepoints(
                observed_data.clone(), observed_tp.clone(), observed_mask.clone(), args.sample_tp)
        else:
            subsampled_data, subsampled_tp, subsampled_mask = \
                observed_data, observed_tp, observed_mask
        out = rec(torch.cat((subsampled_data, subsampled_mask), 2), subsampled_tp)
        qz0_mean = out[:, :, :args.latent_dim]
        qz0_logvar = out[:, :, args.latent_dim:]
        # epsilon = torch.randn(qz0_mean.size()).to(device)
        epsilon = torch.randn(
            args.k_iwae, qz0_mean.shape[0], qz0_mean.shape[1], qz0_mean.shape[2]
        ).to(device)
        z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean
        z0 = z0.view(-1, qz0_mean.shape[1], qz0_mean.shape[2])
        pred_x = dec(
            z0,
            observed_tp[None, :, :].repeat(args.k_iwae, 1, 1).view(-1, observed_tp.shape[1])
        )
        # nsample, batch, seqlen, dim
        pred_x = pred_x.view(args.k_iwae, batch_len, pred_x.shape[1], pred_x.shape[2])
        # compute loss
        logpx, analytic_kl = utils.compute_losses(
            dim, train_batch, qz0_mean, qz0_logvar, pred_x, args, device)
        loss = -(torch.logsumexp(logpx - kl_coef * analytic_kl, dim=0).mean(0) - np.log(args.k_iwae))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_len
        train_n += batch_len
        avg_reconst += torch.mean(logpx) * batch_len
        avg_kl += torch.mean(analytic_kl) * batch_len
        mse += utils.mean_squared_error(
            observed_data, pred_x.mean(0), observed_mask) * batch_len

    print('Iter: {}, avg elbo: {:.4f}, avg reconst: {:.4f}, avg kl: {:.4f}, mse: {:.6f}'
        .format(itr, train_loss / train_n, -avg_reconst / train_n, avg_kl / train_n, mse / train_n))
    writer.add_scalar('avg elbo', train_loss / train_n, itr)
    writer.add_scalar('avg reconst', -avg_reconst / train_n, itr)
    writer.add_scalar('avg kl', avg_kl / train_n, itr)
    writer.add_scalar('avg mse', mse / train_n, itr)

    if itr % 10 == 0:
        mse = utils.evaluate(dim, rec, dec, test_loader, args, 1)
        writer.add_scalar('Test MSE', mse, itr)
        print('Test Mean Squared Error', mse)
    if itr % 10 == 0 and args.save:
        path_save = path + args.dataset + '_' + args.enc + '_' + args.dec + '_' + str(experiment_id) + '.h5'
        torch.save({
            'args': args,
            'epoch': itr,
            'rec_state_dict': rec.state_dict(),
            'dec_state_dict': dec.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': -loss,
        }, path_save)